In [ ]:
import cv2
import mediapipe as mp 
import time

In [ ]:
class FaceDetector():
    def __init__ (self, minDetectionCon=0.5):
        self.minDetectionCon = minDetectionCon

        self.mpFace = mp.solutions.face_detection
        self.mpDraw = mp.solutions.drawing_utils
        self.fd = self.mpFace.FaceDetection(self.minDetectionCon)
        #if(img is not None): 
    
    def findFaces(self, img, draw=True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.fd.process(imgRGB)
        bboxlist = []
        if self.results.detections:
            for id, detection in enumerate(self.results.detections):
                bboxC = detection.location_data.relative_bounding_box # contains xmin,ymin,width,height
                ih, iw, ic = img.shape
                bbox = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                bboxlist.append([bbox, detection.score[0]])
                if draw:
                    #cv2.rectangle(img, bbox, (255,254,254), 2)
                    img = self.fancyDraw(img, bbox)
                # backslash '\' is interpreted as a line-continuation character
                    cv2.putText(img, f'{int(detection.score[0]*100)}%', 
                            (bbox[0],bbox[1]-20), cv2.FONT_HERSHEY_PLAIN, 2, (255,2,254), 2)
                
        return img, bboxlist
    
    def fancyDraw(self, img, bbox, l= 30, t=5, rt=1):
        x, y, w, h = bbox
        x1, y1 = x+w, y+h
        
        cv2.rectangle(img, bbox, (255,254,254), rt)
        # for top left corner
        cv2.line(img, (x,y), (x+l,y), (255,254,254), t) # draws line on top of our bounding box
        cv2.line(img, (x,y), (x,y+l), (255,254,254), t)
        
        # for top right corner
        cv2.line(img, (x1,y), (x1-l,y), (255,254,254), t) # draws line on top of our bounding box
        cv2.line(img, (x1,y), (x1,y+l), (255,254,254), t)
        
         # for bottom left corner
        cv2.line(img, (x,y1), (x+l,y1), (255,254,254), t) # draws line on top of our bounding box
        cv2.line(img, (x,y1), (x,y1-l), (255,254,254), t)
        
        # for top bottom corner
        cv2.line(img, (x1,y1), (x1-l,y1), (255,254,254), t) # draws line on top of our bounding box
        cv2.line(img, (x1,y1), (x1,y1-l), (255,254,254), t)
        
        return img

    

In [ ]:
def main():
    cap = cv2.VideoCapture('faceDetectionvideos/1.mp4')
    pTime = 0
    faceDetect = FaceDetector()
    
    while True:
        success, img = cap.read()
        img, bboxlist = faceDetect.findFaces(img,draw=False)
        print(bboxlist[0])


        cTime = time.time()
        fps = 1/(cTime - pTime)
        pTime = cTime

        cv2.putText(img, f'FPS={int(fps)}', (20,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,2,254), 2)
        cv2.imshow('Image', img)
        cv2.waitKey(10)
    

In [ ]:
if __name__ == '__main__':
    main()